In [1]:
project = 'derivative_project05'
import antropy as ant
from antropy import lziv_complexity
import matplotlib
# matplotlib.use('Qt5Agg')
# %matplotlib

#%matplotlib qt #**

#%matplotlib tk
import matplotlib.pyplot as plt
from matplotlib.pyplot import plot, show
import pathlib
import mne
import os
import os.path as op
from mne import viz
from mne import io
from mne.preprocessing import (ICA, create_eog_epochs, create_ecg_epochs, corrmap)
import zipfile
import sys
import pandas as pd
import numpy as np
from IPython.utils import io
import neurokit2 as nk
import scipy
from scipy import stats
import scipy.stats
import seaborn as sns
import pickle
import itertools
from whatever import *
import time

from scipy.stats import zscore
def standardize(lst):
    mean = sum(lst) / len(lst)
    variance = sum([((x - mean) ** 2) for x in lst]) / len(lst)
    std_dev = variance ** 0.5
    return [(x - mean) / std_dev for x in lst]


from pandas import read_csv
from pandas.plotting import scatter_matrix
import math
import random
import csv

import sklearn
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import sklearn.externals 
import joblib
from sklearn.svm import SVC 
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
#from sklearn.model_selection import groupKFold, GridSearchCV
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
    # from sklearn.metrics import accuracy_score
    # from sklearn.metrics import confusion_matrix
    # from sklearn.metrics import classification_report
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

# pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)
from IPython.display import display

# From ---> 'LZC TimeSeries Feature Extraction-updated (complete)'

## 1. LZc calculation used code that was adapted from Schartner et al., 2015
            - for each participant, and for each of thier segments, calculate normalized LZc
            - for each participant, average the normalized LZc across all segments
            - each participant has 1 value = subj-i(mean(norm(LZc(seg-i))))

#### 1.5.-Prepoc & Feature Extraction
            - Main 
            - dif brain regions (ROIs)
                - TenSecEpochs_19el-1
                - TenSecEpochs_19el-2
                - TenSecEpochs_19el-3
                - TenSecEpochs_19el-4
                - TenSecEpochs_19el-5
            - '10secEpochs'
            - 'TenSecEpochs' : using schartner et al., 2015 code for LZc
            - 'TenSecEpochs_woutreref'
            - 'TenSecEpochs_max' : using from antropy import lziv_complexity
            - 'TenSecEpochs_max_woutreref' 
            - 'TenSecEpochs_mat' : (preproc w/ filter before reref and highass 0.1) i already did this....
            - 'TenSecEpochs_MatandMax' : 

In [ ]:
#v4
# Schartner et al., 2015
from numpy import *
from numpy.linalg import * 
from scipy import signal
from scipy.signal import hilbert
from scipy.stats import ranksums
from scipy.io import savemat
from scipy.io import loadmat
from random import *
from itertools import combinations
from pylab import *
# def Preproc(X):
#     Xp=signal.detrend(X-mean(X), axis=0)
#     return Xp
def str_col(X):
    co=len(X)
    A=abs(hilbert(X))
    TH=mean(A)
    S='' 
    for t in range(co):
        if A[t]>TH:
            S+='1'
        else:
            S+='0'
    return S
def cpr(string):
    d={} 
    w = ''
    i=1
    for c in string:
        wc = w + c
        if wc in d:
            w = wc
        else:
            d[wc]=wc
            w = c
        i+=1
    return len(d)

def LZc(X):
#     Xp = Preproc(X)
    S = str_col(X) 
    Slist = list(S) 
    shuffle(Slist) 
    SS = '' 
    for l in range(len(Slist)): 
        SS += Slist[l]
    return cpr(S)/float(cpr(SS))


# Data
fnames = sorted(os.listdir('./cutdata'))
for f in fnames:
    with io.capture_output() as captured:
        raw = mne.io.read_raw_fif(f'./cutdata/{f}/{f}.fif', preload=True)
        raw.info['bads'] = []
        raw2 = raw.copy().pick_types(eeg = True)
        raw3 = nk.mne_crop(raw2.copy(),smin=0, smax=75500, include_tmax=False)
    with io.capture_output() as captured:
        raw3.notch_filter(60)
        raw3.filter(0.1, 50)
        raw3.drop_channels(new_non_brain_el)
        raw3.set_eeg_reference(ref_channels='average')
    vars()[f] = raw3.copy()
for window in ['10secwindows']:
    print(window)
    create_folder = f'./results/NewDataResults/save/{window}'
    if not os.path.exists(create_folder):
        os.makedirs(create_folder)
        print (f'main folder created: {create_folder}')
    else:
        print("main save folder already exists.")
    # Complexity measure
    m='LZc'
    print ('\n************************',m,'************************\n')
    measure = m
    # create numpy save folder
    create_folder2 = f'./results/NewDataResults/save/{window}/{m}'
    if not os.path.exists(create_folder2):
        os.makedirs(create_folder2)
        print (f'secondary folder created: {create_folder2}')
    else:
        print(f"secondary folder already exists ({m}).")
    svd = lambda x: LZc(x)
    # main dataframe
    vars()[m+window]= pd.DataFrame()
    for f in fnames:
        print ('_____',f, '_____')
        data = vars()[f]
        Elist = []
        if window == '10secwindows':
            # 10secwindows (30 columns)
            for e in data.ch_names:
                print (f'{f}_{e}', end=', ')
                signal = data.get_data(picks = e)[0][500:]
                signal_2d = signal.reshape(-1, 2500)
                svd_2d = np.apply_along_axis(svd, axis=1, arr=signal_2d)
                Elist.append(svd_2d)

        vars()['df_'+f] = pd.DataFrame(Elist)
        ID_with_suffix = [f+'_'+ID for ID in data.ch_names]
        vars()['df_'+f].index = [ID_with_suffix]

        vars()[m+window] = pd.concat([vars()[m+window],vars()['df_'+f]],axis=0) 

        vars()['df_'+f].to_csv(f'./results/NewDataResults/save/{window}/{m}/df_{f}.csv', index=True) 
        print (f'   - df_{f}.csv is saved')
    vars()[m+window].to_csv(f'./results/NewDataResults/save/{window}/{m}/{m}_{window}.csv', index=True)
    print (f'________________________________________________________\n   - saved The Big GUy!!!!! ({m}_{window}.csv)\n________________________________________________________\n')



# NewDataResults
- results
 - NewDataResults
    - LZC
        - TenSecEpochs
            - dfLZC.csv
            - dict_TenSecEpochs.json
    - save
        - 10secwindows
            - svd1
                - svd1_10secwindows.csv
                - df_CCTC03_IT.csv
                - ...
            - svd2
            - approx
            - sample
            - detrended
            - katz
            - cwpen
            - bubben
            - ll
            - mswpen

# LZC
- 'TenSecEpochs' : using schartner et al., 2015 code for LZc

In [2]:
#v2
# Schartner et al., 2015
from numpy import *
from numpy.linalg import * 
from scipy import signal
from scipy.signal import hilbert
from scipy.stats import ranksums
from scipy.io import savemat
from scipy.io import loadmat
from random import *
from itertools import combinations
from pylab import *

# Detrend and normalize input data, X a multidimensional time series
def Preproc(X):
    ro,co=shape(X)
    Xp = zeros((ro,co))
    for i in range(ro):
        Xp[i,:]=signal.detrend(X[i,:]-mean(X[i,:]), axis=0)
    return Xp
###################################################################
### LZc - Lempel-Ziv Complexity, column-by-column concatenation ###
###################################################################
# Binarized input matrix concatenated comlumn-by-column = Sequience
def str_col(Xp):
    # Input: Continuous multidimensional time series
    # Output: One string 
    ro,co=shape(Xp)
    TH=zeros(ro)
    A=zeros((ro,co))
    for i in range(ro):#91 electrodes
        A[i,:]=abs(hilbert(Xp[i,:])) #absolute value of (the Computed analytic signal, (using the Hilbert transform))
        TH[i]=mean(A[i,:]) #Threshold
    S='' # Sequence
    for t in range(co): # for each timepoint
        for i in range(ro): # for each electrode
            if A[i,t]>TH[i]:
                S+='1'
            else:
                S+='0'
    return S

# Lempel-Ziv-Welch compression of binary input string
def cpr(string): #input S = sequence
    # Lempel-Ziv-Welch compression of binary input string, e.g. string='0010101'. 
        #It outputs the size of the dictionary of binary words.
    d={} 
    w = ''
    i=1
    for c in string: # string = sequence 
        wc = w + c
        if wc in d:
            w = wc
        else:
            d[wc]=wc
            w = c
        i+=1
    return len(d)

def LZc(X): # X a multidimensional time series
    # Compute LZc and use shuffled result as normalization
    Xp = Preproc(X) # Detrend and normalize input data, 
    S = str_col(Xp) # Binarized & concatenated comlumn-by-column = Sequience = S
    Slist = list(S) # digits of string(S) --become-> elements in a list
    shuffle(Slist) # shuffle in-place Slist=shuffled
    SS = '' # SS = Shuffled-Sequience 
    for l in range(len(Slist)): 
        SS += Slist[l] # turning Shuffled_Slist back into a string 
    return cpr(S)/float(cpr(SS)) # Lempel-Ziv-Welch compression of binary input string



## Preproc & Feature Extraction
- Main 
- dif brain regions (ROIs)
    - TenSecEpochs_19el-1
    - TenSecEpochs_19el-2
    - TenSecEpochs_19el-3
    - TenSecEpochs_19el-4
    - TenSecEpochs_19el-5


- '10secEpochs'
- 'TenSecEpochs' : using schartner et al., 2015 code for LZc
- 'TenSecEpochs_woutreref'
- 'TenSecEpochs_max' : using from antropy import lziv_complexity
- 'TenSecEpochs_max_woutreref' 
- 'TenSecEpochs_mat' : (preproc w/ filter before reref and highass 0.1) i already did this....
- 'TenSecEpochs_MatandMax' : 

In [3]:
fnames = sorted(os.listdir('./cutdata'))
len(fnames)

276

In [4]:
#NEW DATA 
#MAIN                                                #(Complete)
window = '0dot08SecEpochs' #'TenSecEpochs' #'TenSecEpochs_max_woutreref'#'TenSecEpochs_mat'#'TenSecEpochs_MatandMax' #'TenSecEpochs_woutreref' # window = '10secEpochs'
# create save folders
create_folder = f'./results/NewDataResults/LZC/{window}'
if not os.path.exists(create_folder):
    os.makedirs(create_folder)
    print (f'main folder created: {create_folder}')
else:
    print("main save folder already exists.")

## 10secEpochs / Broadband
fnames = sorted(os.listdir('./cutdata'))
ssfnames = [f for f in fnames if 'HC05_IT1st' not in f]
for f in ssfnames:
    with io.capture_output() as captured:# Load data
        raw = mne.io.read_raw_fif(f'./cutdata/{f}/{f}.fif', preload=True)
        # raw = mne.io.read_raw_fif(f'./cutdata/CCTC03_RS/CCTC03_RS.fif', preload=True)
        raw.info['bads']= []
        raw2 = raw.copy().pick_types(eeg = True)
        raw3 = nk.mne_crop(raw2.copy(),smin=0, smax=75000, include_tmax=False)
    with io.capture_output() as captured:# Preprocessing
        raw3.notch_filter(60)# Filter
        raw3.filter(0.1, 50) #downsample after filtering??????
        raw3.drop_channels(new_non_brain_el)# Remove non-brain channels
        
        
        
        raw3.set_eeg_reference(ref_channels='average')# Re-refrence
        
        
        
        
        
        #vars()['raw_'+f] = raw3.copy() # Saving variable names 
        #data = raw3.copy()# Epoch
        epochs = mne.make_fixed_length_epochs(raw3.copy(), duration=0.08, reject_by_annotation=True, overlap=0.0)#epochs #number of epochs for participants should have a normal distribution
        Aepochs=epochs.get_data() #(30, 91, 2500)... (15000, 91, 5)
    vars()[f] = Aepochs
        
# main dataframe
subResults = [] # Results from each subjects AverageLZc (average across participant's segments)
dict_LZcSegmentsList = {} # LZc values for each segment (NOT averaged)

for f in ssfnames:
    data = vars()[f]
    segments, channels, timepoints = shape(data) #(30, 91, 2500)
    LZcSegmentsList = []
    for i in range(segments):
        X = data[i] 
        LZcSegment = LZc(X)
        LZcSegmentsList.append(LZcSegment)
        #print (f'{i} --- {LZcSegment}')
    AverageLZc = sum(LZcSegmentsList)/len(LZcSegmentsList) #mean(LZcSegmentsList)
    print ('_____',f, '_____', AverageLZc)
    vars()['AverageLZC_'+f] = AverageLZc
    subResults.append(AverageLZc)
    vars()['LZcSegmentsList_'+f] = LZcSegmentsList
    dict_LZcSegmentsList[f] = LZcSegmentsList

dfLZC = pd.DataFrame(subResults, ssfnames)#, ['LZC_values'])
# dfLZC = pd.DataFrame(subResults)
# dfLZC.index = [ssfnames]


# save
dfLZC.to_csv(f'./results/NewDataResults/LZC/{window}/dfLZC.csv', index=True)  
print ('   - dfLZC.csv is saved')
display(dfLZC)
# save/write dict
import json
with open(f'./results/NewDataResults/LZC/{window}/dict_{window}.json', 'w') as file:
    json.dump(dict_LZcSegmentsList, file)
    
    

main save folder already exists.
_____ CCTC03_IT _____ 0.9497973103672804
_____ CCTC03_RS _____ 0.947136465131423
_____ CCTC03_ST _____ 0.9375819746220097
_____ CCTC03part2_IT _____ 0.9685464600543233
_____ CCTC03part2_RS _____ 0.915221217870828
_____ CCTC03part2_ST _____ 0.968019187452015
_____ CCTC04_IT _____ 0.9672642768503988
_____ CCTC04_RS _____ 0.9575073827474365
_____ CCTC04_RSredo _____ 0.9483748478997612
_____ CCTC04_ST _____ 0.9661086194534237
_____ CCTC05_IT _____ 0.9363745939425333
_____ CCTC05_RS _____ 0.9195532560389642
_____ CCTC05_ST _____ 0.9264768960919513
_____ CCTC07NETICUsedoff_IT _____ 0.9021788725698712
_____ CCTC07NETICUsedoff_RS _____ 0.8896232852681621
_____ CCTC07NETICUsedoff_ST _____ 0.8849666347293119
_____ CCTC07NETICUsedon1_IT _____ 0.99053114867292
_____ CCTC07NETICUsedon1_RS _____ 0.9932681972439955
_____ CCTC07NETICUsedon1_ST _____ 0.9864950202003829
_____ CCTC07_IT _____ 0.8762699384997243
_____ CCTC07_RS _____ 0.8452693455288783
_____ CCTC07_ST ____

_____ NETICU17sedoff_ST _____ 0.9285722493928384
_____ NETICU17sedon1_IT _____ 0.8999569006154172
_____ NETICU17sedon1_RS _____ 0.9015026685562592
_____ NETICU17sedon1_ST _____ 0.8910938823792716
_____ NETICU18sedoff_RS _____ 0.9308529224069301
_____ NETICU18sedon1_IT _____ 0.8769721742974528
_____ NETICU18sedon1_RS _____ 0.944238494208667
_____ NETICU18sedon1_ST _____ 0.908562626159079
_____ NETICU19sedoff_IT _____ 0.9256386978752017
_____ NETICU19sedoff_RS _____ 0.9498911163124961
_____ NETICU19sedoff_ST _____ 0.8779291528504231
_____ NETICU19sedon1_IT _____ 0.9602617921447701
_____ NETICU19sedon1_RS _____ 0.9643715944335645
_____ NETICU19sedon1_ST _____ 0.9570453741221581
_____ NETICU20sedoff_RS _____ 0.8126691573493527
_____ NETICU20sedon1_IT _____ 0.6910562583579527
_____ NETICU20sedon1_RS _____ 0.7222146212436569
_____ NETICU20sedon1_ST _____ 0.7888470823852392
_____ NETICU22sedoff_IT _____ 0.8474818034004381
_____ NETICU22sedoff_RS _____ 0.8463395562866137
_____ NETICU22sedoff_S

,0
CCTC03_IT,0.949797
CCTC03_RS,0.947136
CCTC03_ST,0.937582
CCTC03part2_IT,0.968546
CCTC03part2_RS,0.915221
...,...
NETICU41sedon2_ST,0.901282
NETICU46sedoff_RS,0.879240
NETICU46sedon1_IT,0.877750
NETICU46sedon1_RS,0.879071


In [ ]:
#NEW DATA 
#MAIN                                                #(Complete)
#window = 'TenSecEpochs_max' #'TenSecEpochs' #'TenSecEpochs_max_woutreref'#'TenSecEpochs_mat'#'TenSecEpochs_MatandMax' #'TenSecEpochs_woutreref' # window = '10secEpochs'
#create save folders
for  window in ['0dot2SecEpochs','1SecEpochs','5SecEpochs']:
    create_folder = f'./results/NewDataResults/LZC/{window}'
    if not os.path.exists(create_folder):
        os.makedirs(create_folder)
        print (f'main folder created: {create_folder}')
    else:
        print("main save folder already exists.")
## 10secEpochs / Broadband
fnames = sorted(os.listdir('./cutdata'))
ssfnames = [f for f in fnames if 'HC05_IT1st' not in f]
for f in ssfnames:
    with io.capture_output() as captured:# Load data
        raw = mne.io.read_raw_fif(f'./cutdata/{f}/{f}.fif', preload=True)
        # raw = mne.io.read_raw_fif(f'./cutdata/CCTC03_RS/CCTC03_RS.fif', preload=True)
        raw.info['bads']= []
        raw2 = raw.copy().pick_types(eeg = True)
        raw3 = nk.mne_crop(raw2.copy(),smin=0, smax=75000, include_tmax=False)
    with io.capture_output() as captured:# Preprocessing
        raw3.notch_filter(60)# Filter
        raw3.filter(0.1, 50) #downsample after filtering??????
        raw3.drop_channels(new_non_brain_el)# Remove non-brain channels
        
        
        
        raw3.set_eeg_reference(ref_channels='average')# Re-refrence
        
        
        
        
        
        #vars()['raw_'+f] = raw3.copy() # Saving variable names 
        #data = raw3.copy()# Epoch
        epochs1 = mne.make_fixed_length_epochs(raw3.copy(), duration=0.2, reject_by_annotation=True, overlap=0.0)#epochs #number of epochs for participants should have a normal distribution
        Aepochs1=epochs1.get_data() #(30, 91, 2500)
        epochs2 = mne.make_fixed_length_epochs(raw3.copy(), duration=1, reject_by_annotation=True, overlap=0.0)#epochs #number of epochs for participants should have a normal distribution
        Aepochs2=epochs2.get_data() #(30, 91, 2500)
        epochs3 = mne.make_fixed_length_epochs(raw3.copy(), duration=5, reject_by_annotation=True, overlap=0.0)#epochs #number of epochs for participants should have a normal distribution
        Aepochs3=epochs3.get_data() #(30, 91, 2500)
    vars()[f] = Aepochs1
    vars()[f+'_1SecEpochs'] = Aepochs2
    vars()[f+'_5SecEpochs'] = Aepochs3
    
    
    
# main dataframe
subResults1 = [] # Results from each subjects AverageLZc (average across participant's segments)
subResults2 = []
subResults3 = []
dict_LZcSegmentsList1 = {} # LZc values for each segment (NOT averaged)
dict_LZcSegmentsList2 = {}
dict_LZcSegmentsList3 = {} 
#for  window in ['0dot2SecEpochs','1SecEpochs','5SecEpochs']:
for f in ssfnames:
    data1 = vars()[f]
    data2 = vars()[f+'_1SecEpochs']
    data3 = vars()[f+'_5SecEpochs']
    segments1, channels1, timepoints1 = shape(data1) #(30, 91, 2500)
    segments2, channels2, timepoints2 = shape(data2) #(30, 91, 2500)
    segments3, channels3, timepoints3 = shape(data3) #(30, 91, 2500)
    LZcSegmentsList1 = []
    LZcSegmentsList2 = []
    LZcSegmentsList3 = []
    for i in range(segments1):
        X = data1[i] 
        LZcSegment = LZc(X)
        LZcSegmentsList1.append(LZcSegment)
        #print (f'{i} --- {LZcSegment}')
    for i in range(segments2):
        X = data2[i] 
        LZcSegment = LZc(X)
        LZcSegmentsList2.append(LZcSegment)
    for i in range(segments3):
        X = data3[i] 
        LZcSegment = LZc(X)
        LZcSegmentsList3.append(LZcSegment)
    AverageLZc1 = sum(LZcSegmentsList1)/len(LZcSegmentsList1) #mean(LZcSegmentsList)
    AverageLZc2 = sum(LZcSegmentsList2)/len(LZcSegmentsList2) #mean(LZcSegmentsList)
    AverageLZc3 = sum(LZcSegmentsList3)/len(LZcSegmentsList3) #mean(LZcSegmentsList)
    print ('_____',f, '_____', AverageLZc1)
    vars()['AverageLZC_'+f] = AverageLZc1
    subResults1.append(AverageLZc1)
    subResults2.append(AverageLZc2)
    subResults3.append(AverageLZc3)
    dict_LZcSegmentsList1[f] = LZcSegmentsList1
    dict_LZcSegmentsList2[f] = LZcSegmentsList2
    dict_LZcSegmentsList3[f] = LZcSegmentsList3
dfLZC1 = pd.DataFrame(subResults1, ssfnames)
dfLZC2 = pd.DataFrame(subResults2, ssfnames)
dfLZC3 = pd.DataFrame(subResults3, ssfnames)


# save
dfLZC1.to_csv(f'./results/NewDataResults/LZC/0dot2SecEpochs/dfLZC.csv', index=True)  
dfLZC2.to_csv(f'./results/NewDataResults/LZC/1SecEpochs/dfLZC.csv', index=True)  
dfLZC3.to_csv(f'./results/NewDataResults/LZC/{window}/dfLZC.csv', index=True)  
print ('   - dfLZC.csv is saved')
display(dfLZC1)
# save/write dict
import json
with open(f'./results/NewDataResults/LZC/0dot2SecEpochs/dict_0dot2SecEpochs.json', 'w') as file:
    json.dump(dict_LZcSegmentsList1, file)
with open(f'./results/NewDataResults/LZC/1SecEpochs/dict_1SecEpochs.json', 'w') as file:
    json.dump(dict_LZcSegmentsList2, file)
with open(f'./results/NewDataResults/LZC/5SecEpochs/dict_5SecEpochs.json', 'w') as file:
    json.dump(dict_LZcSegmentsList3, file)


    


In [ ]:
# FREQUINCY BANDS----TenSecEpochs
            # -Dif Frequincy bands 
            # -from derivative project 03 - cyan
print ('FREQUINCY_BANDS_TenSecEpochs')

####################
#NEW DATA  AND FREQUENCY
#MAIN                                                #(Complete)
#window = 'TenSecEpochs_max' #'TenSecEpochs' #'TenSecEpochs_max_woutreref'#'TenSecEpochs_mat'#'TenSecEpochs_MatandMax' #'TenSecEpochs_woutreref' # window = '10secEpochs'
#create save folders
for  window in ['TenSecEpochs_delta','TenSecEpochs_theta','TenSecEpochs_alpha','TenSecEpochs_beta','TenSecEpochs_gamma']:
    create_folder = f'./results/NewDataResults/LZC/{window}'
    if not os.path.exists(create_folder):
        os.makedirs(create_folder)
        print (f'main folder created: {create_folder}')
    else:
        print("main save folder already exists.")
## 10secEpochs / Broadband
fnames = sorted(os.listdir('./cutdata'))
ssfnames = [f for f in fnames if 'HC05_IT1st' not in f]
for f in ssfnames:
    with io.capture_output() as captured:# Load data
        raw = mne.io.read_raw_fif(f'./cutdata/{f}/{f}.fif', preload=True)
        raw.info['bads']= []
        raw2 = raw.copy().pick_types(eeg = True)
        raw3 = nk.mne_crop(raw2.copy(),smin=0, smax=75000, include_tmax=False)
    with io.capture_output() as captured:# Preprocessing
        raw3.notch_filter(60)# Filter
        raw3.filter(0.1, 50) #downsample after filtering??????
        raw3.drop_channels(new_non_brain_el)# Remove non-brain channels
        delta = raw3.copy().filter(1, 4)#Delta waves (0.5–4 Hz)
        theta = raw3.copy().filter(4, 8)#Theta waves (4–8 Hz)
        alpha = raw3.copy().filter(8, 12)#Alpha waves (8–13 Hz)
        beta = raw3.copy().filter(12, 30)#Beta waves (13–35 Hz)
        gamma = raw3.copy().filter(30, 40)
        


        
        
        
        # Re-refrence
        delta.set_eeg_reference(ref_channels='average')
        theta.set_eeg_reference(ref_channels='average')
        alpha.set_eeg_reference(ref_channels='average')
        beta.set_eeg_reference(ref_channels='average')
        gamma.set_eeg_reference(ref_channels='average')
        
        
        
        
        #vars()['raw_'+f] = raw3.copy() # Saving variable names 
        #data = raw3.copy()# Epoch
        epochs1 = mne.make_fixed_length_epochs(delta.copy(), duration=10, reject_by_annotation=True, overlap=0.0)#epochs #number of epochs for participants should have a normal distribution
        Aepochs1=epochs1.get_data() #(30, 91, 2500)
        epochs2 = mne.make_fixed_length_epochs(theta.copy(), duration=10, reject_by_annotation=True, overlap=0.0)#epochs #number of epochs for participants should have a normal distribution
        Aepochs2=epochs2.get_data() #(30, 91, 2500)
        epochs3 = mne.make_fixed_length_epochs(alpha.copy(), duration=10, reject_by_annotation=True, overlap=0.0)#epochs #number of epochs for participants should have a normal distribution
        Aepochs3=epochs3.get_data() #(30, 91, 2500)
        epochs4 = mne.make_fixed_length_epochs(beta.copy(), duration=10, reject_by_annotation=True, overlap=0.0)#epochs #number of epochs for participants should have a normal distribution
        Aepochs4=epochs4.get_data() #(30, 91, 2500)
        epochs5 = mne.make_fixed_length_epochs(gamma.copy(), duration=10, reject_by_annotation=True, overlap=0.0)#epochs #number of epochs for participants should have a normal distribution
        Aepochs5=epochs5.get_data() #(30, 91, 2500)
    vars()[f] = Aepochs1
    vars()[f+'_1SecEpochs'] = Aepochs2
    vars()[f+'_5SecEpochs'] = Aepochs3
    vars()[f+'_TenSecEpochs_beta'] = Aepochs4
    vars()[f+'_TenSecEpochs_gamma'] = Aepochs5
    
# main dataframe
subResults1 = [] # Results from each subjects AverageLZc (average across participant's segments)
subResults2 = []
subResults3 = []
subResults4 = []
subResults5 = []
dict_LZcSegmentsList1 = {} # LZc values for each segment (NOT averaged)
dict_LZcSegmentsList2 = {}
dict_LZcSegmentsList3 = {}
dict_LZcSegmentsList4 = {}
dict_LZcSegmentsList5 = {} 
#for  window in ['0dot2SecEpochs','1SecEpochs','5SecEpochs']:
for f in ssfnames:
    data1 = vars()[f]
    data2 = vars()[f+'_1SecEpochs']
    data3 = vars()[f+'_5SecEpochs']
    data4 = vars()[f+'_TenSecEpochs_beta']
    data5 = vars()[f+'_TenSecEpochs_gamma']
    segments1, channels1, timepoints1 = shape(data1) #(30, 91, 2500)
    segments2, channels2, timepoints2 = shape(data2) #(30, 91, 2500)
    segments3, channels3, timepoints3 = shape(data3) #(30, 91, 2500)
    segments4, channels4, timepoints4 = shape(data4) #(30, 91, 2500)
    segments5, channels5, timepoints5 = shape(data5) #(30, 91, 2500)
    LZcSegmentsList1 = []
    LZcSegmentsList2 = []
    LZcSegmentsList3 = []
    LZcSegmentsList4 = []
    LZcSegmentsList5 = []
    for i in range(segments1):
        X = data1[i] 
        LZcSegment = LZc(X)
        LZcSegmentsList1.append(LZcSegment)
        #print (f'{i} --- {LZcSegment}')
    for i in range(segments2):
        X = data2[i] 
        LZcSegment = LZc(X)
        LZcSegmentsList2.append(LZcSegment)
    for i in range(segments3):
        X = data3[i] 
        LZcSegment = LZc(X)
        LZcSegmentsList3.append(LZcSegment)
    for i in range(segments4):
        X = data4[i] 
        LZcSegment = LZc(X)
        LZcSegmentsList4.append(LZcSegment)
    for i in range(segments5):
        X = data5[i] 
        LZcSegment = LZc(X)
        LZcSegmentsList5.append(LZcSegment)
    AverageLZc1 = sum(LZcSegmentsList1)/len(LZcSegmentsList1) #mean(LZcSegmentsList)
    AverageLZc2 = sum(LZcSegmentsList2)/len(LZcSegmentsList2) #mean(LZcSegmentsList)
    AverageLZc3 = sum(LZcSegmentsList3)/len(LZcSegmentsList3) #mean(LZcSegmentsList)
    AverageLZc4 = sum(LZcSegmentsList4)/len(LZcSegmentsList4) #mean(LZcSegmentsList)
    AverageLZc5 = sum(LZcSegmentsList5)/len(LZcSegmentsList5) #mean(LZcSegmentsList)
    print ('_____',f, '_____', AverageLZc1)
    vars()['AverageLZC_'+f] = AverageLZc1
    subResults1.append(AverageLZc1)
    subResults2.append(AverageLZc2)
    subResults3.append(AverageLZc3)
    subResults4.append(AverageLZc4)
    subResults5.append(AverageLZc5)
    dict_LZcSegmentsList1[f] = LZcSegmentsList1
    dict_LZcSegmentsList2[f] = LZcSegmentsList2
    dict_LZcSegmentsList3[f] = LZcSegmentsList3
    dict_LZcSegmentsList4[f] = LZcSegmentsList4
    dict_LZcSegmentsList5[f] = LZcSegmentsList5
dfLZC1 = pd.DataFrame(subResults1, ssfnames)
dfLZC2 = pd.DataFrame(subResults2, ssfnames)
dfLZC3 = pd.DataFrame(subResults3, ssfnames)
dfLZC4 = pd.DataFrame(subResults4, ssfnames)
dfLZC5 = pd.DataFrame(subResults5, ssfnames)

# save
dfLZC1.to_csv(f'./results/NewDataResults/LZC/TenSecEpochs_delta/dfLZC.csv', index=True)  
dfLZC2.to_csv(f'./results/NewDataResults/LZC/TenSecEpochs_theta/dfLZC.csv', index=True)  
dfLZC3.to_csv(f'./results/NewDataResults/LZC/TenSecEpochs_alpha/dfLZC.csv', index=True)  
dfLZC4.to_csv(f'./results/NewDataResults/LZC/TenSecEpochs_beta/dfLZC.csv', index=True)  
dfLZC5.to_csv(f'./results/NewDataResults/LZC/TenSecEpochs_gamma/dfLZC.csv', index=True) 
print ('   - dfLZC.csv is saved')
display(dfLZC1)
# save/write dict
import json
with open(f'./results/NewDataResults/LZC/TenSecEpochs_delta/dict_TenSecEpochs_delta.json', 'w') as file:
    json.dump(dict_LZcSegmentsList1, file)
with open(f'./results/NewDataResults/LZC/TenSecEpochs_theta/dict_TenSecEpochs_theta.json', 'w') as file:
    json.dump(dict_LZcSegmentsList2, file)
with open(f'./results/NewDataResults/LZC/TenSecEpochs_alpha/dict_TenSecEpochs_alpha.json', 'w') as file:
    json.dump(dict_LZcSegmentsList3, file)
with open(f'./results/NewDataResults/LZC/TenSecEpochs_beta/dict_TenSecEpochs_beta.json', 'w') as file:
    json.dump(dict_LZcSegmentsList4, file)
with open(f'./results/NewDataResults/LZC/TenSecEpochs_gamma/dict_TenSecEpochs_gamma.json', 'w') as file:
    json.dump(dict_LZcSegmentsList5, file)



In [ ]:
# FREQUINCY BANDS----0dot02SecEpochs
            # -Dif Frequincy bands 
            # -from derivative project 03 - cyan
print ('FREQUINCY_BANDS for 0.02-SecEpochs')

####################
#NEW DATA  AND FREQUENCY
#MAIN                                                #(Complete)
#window = 'TenSecEpochs_max' #'TenSecEpochs' #'TenSecEpochs_max_woutreref'#'TenSecEpochs_mat'#'TenSecEpochs_MatandMax' #'TenSecEpochs_woutreref' # window = '10secEpochs'
#create save folders
for  window in ['0dot02SecEpochs_delta','0dot02SecEpochs_theta','0dot02SecEpochs_alpha','0dot02SecEpochs_beta','0dot02SecEpochs_gamma']:
    create_folder = f'./results/NewDataResults/LZC/{window}'
    if not os.path.exists(create_folder):
        os.makedirs(create_folder)
        print (f'main folder created: {create_folder}')
    else:
        print("main save folder already exists.")
## 10secEpochs / Broadband
fnames = sorted(os.listdir('./cutdata'))
ssfnames = [f for f in fnames if 'HC05_IT1st' not in f]
for f in ssfnames:
    with io.capture_output() as captured:# Load data
        raw = mne.io.read_raw_fif(f'./cutdata/{f}/{f}.fif', preload=True)
        raw.info['bads']= []
        raw2 = raw.copy().pick_types(eeg = True)
        raw3 = nk.mne_crop(raw2.copy(),smin=0, smax=75000, include_tmax=False)
    with io.capture_output() as captured:# Preprocessing
        raw3.notch_filter(60)# Filter
        raw3.filter(0.1, 50) #downsample after filtering??????
        raw3.drop_channels(new_non_brain_el)# Remove non-brain channels
        delta = raw3.copy().filter(1, 4)
        theta = raw3.copy().filter(4, 8)
        alpha = raw3.copy().filter(8, 12)
        beta = raw3.copy().filter(12, 30)
        gamma = raw3.copy().filter(30, 40)
        


        
        
        
        # Re-refrence
        delta.set_eeg_reference(ref_channels='average')
        theta.set_eeg_reference(ref_channels='average')
        alpha.set_eeg_reference(ref_channels='average')
        beta.set_eeg_reference(ref_channels='average')
        gamma.set_eeg_reference(ref_channels='average')
        
        
        
        
        #vars()['raw_'+f] = raw3.copy() # Saving variable names 
        #data = raw3.copy()# Epoch
        epochs1 = mne.make_fixed_length_epochs(delta.copy(), duration=0.02, reject_by_annotation=True, overlap=0.0)#epochs #number of epochs for participants should have a normal distribution
        Aepochs1=epochs1.get_data() #(30, 91, 2500)
        epochs2 = mne.make_fixed_length_epochs(theta.copy(), duration=0.02, reject_by_annotation=True, overlap=0.0)#epochs #number of epochs for participants should have a normal distribution
        Aepochs2=epochs2.get_data() #(30, 91, 2500)
        epochs3 = mne.make_fixed_length_epochs(alpha.copy(), duration=0.02, reject_by_annotation=True, overlap=0.0)#epochs #number of epochs for participants should have a normal distribution
        Aepochs3=epochs3.get_data() #(30, 91, 2500)
        epochs4 = mne.make_fixed_length_epochs(beta.copy(), duration=0.02, reject_by_annotation=True, overlap=0.0)#epochs #number of epochs for participants should have a normal distribution
        Aepochs4=epochs4.get_data() #(30, 91, 2500)
        epochs5 = mne.make_fixed_length_epochs(gamma.copy(), duration=0.02, reject_by_annotation=True, overlap=0.0)#epochs #number of epochs for participants should have a normal distribution
        Aepochs5=epochs5.get_data() #(30, 91, 2500)
    vars()[f] = Aepochs1
    vars()[f+'_1SecEpochs'] = Aepochs2
    vars()[f+'_5SecEpochs'] = Aepochs3
    vars()[f+'_TenSecEpochs_beta'] = Aepochs4
    vars()[f+'_TenSecEpochs_gamma'] = Aepochs5
    
# main dataframe
subResults1 = [] # Results from each subjects AverageLZc (average across participant's segments)
subResults2 = []
subResults3 = []
subResults4 = []
subResults5 = []
dict_LZcSegmentsList1 = {} # LZc values for each segment (NOT averaged)
dict_LZcSegmentsList2 = {}
dict_LZcSegmentsList3 = {}
dict_LZcSegmentsList4 = {}
dict_LZcSegmentsList5 = {} 
#for  window in ['0dot2SecEpochs','1SecEpochs','5SecEpochs']:
for f in ssfnames:
    data1 = vars()[f]
    data2 = vars()[f+'_1SecEpochs']
    data3 = vars()[f+'_5SecEpochs']
    data4 = vars()[f+'_TenSecEpochs_beta']
    data5 = vars()[f+'_TenSecEpochs_gamma']
    segments1, channels1, timepoints1 = shape(data1) #(30, 91, 2500)
    segments2, channels2, timepoints2 = shape(data2) #(30, 91, 2500)
    segments3, channels3, timepoints3 = shape(data3) #(30, 91, 2500)
    segments4, channels4, timepoints4 = shape(data4) #(30, 91, 2500)
    segments5, channels5, timepoints5 = shape(data5) #(30, 91, 2500)
    LZcSegmentsList1 = []
    LZcSegmentsList2 = []
    LZcSegmentsList3 = []
    LZcSegmentsList4 = []
    LZcSegmentsList5 = []
    for i in range(segments1):
        X = data1[i] 
        LZcSegment = LZc(X)
        LZcSegmentsList1.append(LZcSegment)
        #print (f'{i} --- {LZcSegment}')
    for i in range(segments2):
        X = data2[i] 
        LZcSegment = LZc(X)
        LZcSegmentsList2.append(LZcSegment)
    for i in range(segments3):
        X = data3[i] 
        LZcSegment = LZc(X)
        LZcSegmentsList3.append(LZcSegment)
    for i in range(segments4):
        X = data4[i] 
        LZcSegment = LZc(X)
        LZcSegmentsList4.append(LZcSegment)
    for i in range(segments5):
        X = data5[i] 
        LZcSegment = LZc(X)
        LZcSegmentsList5.append(LZcSegment)
    AverageLZc1 = sum(LZcSegmentsList1)/len(LZcSegmentsList1) #mean(LZcSegmentsList)
    AverageLZc2 = sum(LZcSegmentsList2)/len(LZcSegmentsList2) #mean(LZcSegmentsList)
    AverageLZc3 = sum(LZcSegmentsList3)/len(LZcSegmentsList3) #mean(LZcSegmentsList)
    AverageLZc4 = sum(LZcSegmentsList4)/len(LZcSegmentsList4) #mean(LZcSegmentsList)
    AverageLZc5 = sum(LZcSegmentsList5)/len(LZcSegmentsList5) #mean(LZcSegmentsList)
    print ('_____',f, '_____', AverageLZc1)
    vars()['AverageLZC_'+f] = AverageLZc1
    subResults1.append(AverageLZc1)
    subResults2.append(AverageLZc2)
    subResults3.append(AverageLZc3)
    subResults4.append(AverageLZc4)
    subResults5.append(AverageLZc5)
    dict_LZcSegmentsList1[f] = LZcSegmentsList1
    dict_LZcSegmentsList2[f] = LZcSegmentsList2
    dict_LZcSegmentsList3[f] = LZcSegmentsList3
    dict_LZcSegmentsList4[f] = LZcSegmentsList4
    dict_LZcSegmentsList5[f] = LZcSegmentsList5
dfLZC1 = pd.DataFrame(subResults1, ssfnames)
dfLZC2 = pd.DataFrame(subResults2, ssfnames)
dfLZC3 = pd.DataFrame(subResults3, ssfnames)
dfLZC4 = pd.DataFrame(subResults4, ssfnames)
dfLZC5 = pd.DataFrame(subResults5, ssfnames)

# save
dfLZC1.to_csv(f'./results/NewDataResults/LZC/0dot02SecEpochs_delta/dfLZC.csv', index=True)  
dfLZC2.to_csv(f'./results/NewDataResults/LZC/0dot02SecEpochs_theta/dfLZC.csv', index=True)  
dfLZC3.to_csv(f'./results/NewDataResults/LZC/0dot02SecEpochs_alpha/dfLZC.csv', index=True)  
dfLZC4.to_csv(f'./results/NewDataResults/LZC/0dot02SecEpochs_beta/dfLZC.csv', index=True)  
dfLZC5.to_csv(f'./results/NewDataResults/LZC/0dot02SecEpochs_gamma/dfLZC.csv', index=True) 
print ('   - dfLZC.csv is saved')
display(dfLZC1)
# save/write dict
import json
with open(f'./results/NewDataResults/LZC/0dot02SecEpochs_delta/dict_0dot02SecEpochs_delta.json', 'w') as file:
    json.dump(dict_LZcSegmentsList1, file)
with open(f'./results/NewDataResults/LZC/0dot02SecEpochs_theta/dict_0dot02SecEpochs_theta.json', 'w') as file:
    json.dump(dict_LZcSegmentsList2, file)
with open(f'./results/NewDataResults/LZC/0dot02SecEpochs_alpha/dict_0dot02SecEpochs_alpha.json', 'w') as file:
    json.dump(dict_LZcSegmentsList3, file)
with open(f'./results/NewDataResults/LZC/0dot02SecEpochs_beta/dict_0dot02SecEpochs_beta.json', 'w') as file:
    json.dump(dict_LZcSegmentsList4, file)
with open(f'./results/NewDataResults/LZC/0dot02SecEpochs_gamma/dict_0dot02SecEpochs_gamma.json', 'w') as file:
    json.dump(dict_LZcSegmentsList5, file)



### Loading data


In [ ]:
# loading LZC values
window = 'TenSecEpochs' # 'TenSecEpochs_max' # window = '10secEpochs'
vars()[window]= pd.read_csv(f'./results/NewDataResults/LZC/{window}/dfLZC.csv')
vars()[window].rename(columns={vars()[window].columns[0]: 'ID'}, inplace=True)
vars()[window].rename(columns={vars()[window].columns[1]: 'LZC'}, inplace=True)
vars()[window].set_index('ID', inplace=True)
df = vars()[window].copy()
display(df)


# load/read dict
import json
with open(f'./results/NewDataResults/LZC/{window}/dict_{window}.json', 'r') as file:
    loaded_dict = json.load(file)
segsdf = pd.DataFrame(loaded_dict)
display(segsdf)

In [ ]:
print ('hi')

# TimeSeries Feature Extraction-updated (complete)
# Oct.11,2023

Added dif time window options for feature extraction
- 10_sec_windows
- 5_sec_windows
- 2_sec_windows


for 10['svd1', 'svd2', 'approx', 'sample', 'detrended', 'katz', 'cwpen', 'bubben', 'll', 'mswpen']

In [ ]:
#Done Done Done Done Done Done Done Done Done Done Done Done Done Done Done Done Done Done Done Done Done Done Done Done 
#oct.11,2023=================================================================================================#

# Loading and Preprocessing Data
fnames = sorted(os.listdir('./cutdata'))
for f in fnames:
    with io.capture_output() as captured:# Load data
        raw = mne.io.read_raw_fif(f'./cutdata/{f}/{f}.fif', preload=True)
        raw.info['bads'] = []# Keep EEG only (will remove bads too; so I reset bads first)
        raw2 = raw.copy().pick_types(eeg = True)# Crop from 312sec to 302seconds
        raw3 = nk.mne_crop(raw2.copy(),smin=0, smax=75500, include_tmax=False)
    with io.capture_output() as captured:# Preprocessing
        raw3.notch_filter(60)# Filter
        raw3.filter(0.1, 50)
        raw3.drop_channels(new_non_brain_el)# Remove non-brain channels
        raw3.set_eeg_reference(ref_channels='average')# Re-refrence
    vars()[f] = raw3.copy()# Saving variable names 

###########################################
for window in ['5secwindows']:#['2secwindows','5secwindows']: #['2secwindows','5secwindows','10secwindows']: ##<<<<<------------------------set the folder name
###########################################
    # create save folders
    print ("\n=====================================================================")
    print(window)
    print ("=====================================================================")
    create_folder = f'./results/NewDataResults/save/{window}'
    if not os.path.exists(create_folder):
        os.makedirs(create_folder)
        print (f'main folder created: {create_folder}')
    else:
        print("main save folder already exists.")
    # Participant group & Complexity measure
    for m in ['approx', 'sample', 'detrended', 'katz', 'cwpen', 'bubben', 'll', 'mswpen']:#['svd1', 'svd2', 'approx', 'sample', 'detrended', 'katz', 'cwpen', 'bubben', 'll', 'mswpen']: #['svd1', 'svd2', 'approx']:#, 'sample', 'detrended', 'katz', 'cwpen', 'bubben', 'll', 'mswpen']:
        print ('\n************************',m,'************************\n')
        measure = m
        # create numpy save folder
        create_folder2 = f'./results/NewDataResults/save/{window}/{m}'
        if not os.path.exists(create_folder2):
            os.makedirs(create_folder2)
            print (f'secondary folder created: {create_folder2}')
        else:
            print(f"secondary folder already exists ({m}).")

        if m == 'svd1':
            svd = lambda x: nk.entropy_svd(x, delay=1, dimension=2, show=False)[0]
        elif m == 'svd2':
            svd = lambda x: nk.entropy_svd(x, delay=4, dimension=30, show=False)[0]
        elif m == 'approx':
            svd = lambda x: ant.app_entropy(x)
        elif m == 'sample':
            svd = lambda x: ant.sample_entropy(x)
        elif m == 'detrended':
            svd = lambda x: ant.detrended_fluctuation(x)
        elif m == 'katz':
            svd = lambda x: nk.fractal_katz(x)[0]
        elif m == 'cwpen':
            svd = lambda x: nk.entropy_permutation(x, delay=1, dimension=3, corrected=True, weighted=True, conditional=True)[0]
        elif m == 'bubben':
            svd = lambda x: nk.entropy_bubble(x, delay=1, dimension=3, alpha=2)[0]   
        elif m == 'll':
            svd = lambda x: nk.fractal_linelength(x)[0]
        elif m == 'mswpen':
            svd = lambda x: nk.entropy_multiscale(x, scale='default', dimension=3, tolerance='sd', method="MSWPEn", show=False)[0]        



        # main dataframe
        vars()[m+window]= pd.DataFrame()
        for f in fnames:
            print ('_____',f, '_____')
            data = vars()[f]
            Elist = []
            if window == '10secwindows':
                # 10secwindows (30 columns)
                for e in data.ch_names:
                    print (f'{f}_{e}', end=', ')
                    signal = data.get_data(picks = e)[0][500:] #for avg channels --> #signal = data.pick_channels(e).get_data().mean(axis = 0)#[500:]
                    signal_2d = signal.reshape(-1, 2500) #signal_2d = signal.reshape(-1, 1250)
                    svd_2d = np.apply_along_axis(svd, axis=1, arr=signal_2d)
                    Elist.append(svd_2d)
            if window == '5secwindows':
                # 5secwindows (60 columns)
                for e in data.ch_names:
                    print (f'{f}_{e}', end=', ')
                    signal = data.get_data(picks = e)[0][500:]
                    signal_2d = signal.reshape(-1, 1250)
                    svd_2d = np.apply_along_axis(svd, axis=1, arr=signal_2d)
                    Elist.append(svd_2d)
            if window == '2secwindows':
                # 2secwindows (151 columns)
                for e in data.ch_names:
                    print (f'{f}_{e}', end=', ')
                    signal = data.get_data(picks = e)[0]
                    signal_2d = signal.reshape(-1, 500)
                    svd_2d = np.apply_along_axis(svd, axis=1, arr=signal_2d)
                    Elist.append(svd_2d)
            #subject-specific dataframe
            vars()['df_'+f] = pd.DataFrame(Elist)
            ID_with_suffix = [f+'_'+ID for ID in data.ch_names]
            vars()['df_'+f].index = [ID_with_suffix]
            #MAIN
            vars()[m+window] = pd.concat([vars()[m+window],vars()['df_'+f]],axis=0) 


            # save
            vars()['df_'+f].to_csv(f'./results/NewDataResults/save/{window}/{m}/df_{f}.csv', index=True) 
            print (f'   - df_{f}.csv is saved')
        vars()[m+window].to_csv(f'./results/NewDataResults/save/{window}/{m}/{m}_{window}.csv', index=True)
        print (f'________________________________________________________\n   - saved The Big GUy!!!!! ({m}_{window}.csv)\n________________________________________________________\n')


In [ ]:
# windowed complexity values
window = '10secwindows'
for i in ['svd1', 'svd2', 'approx', 'sample', 'detrended', 'katz', 'cwpen', 'bubben', 'll', 'mswpen']:#['approx']:['ll','mswpen','bubben','cwpen']:#['svd2','svd1','approx','sample','detrended','katz']
    vars()[i+'_'+window]= pd.read_csv(f'./results/NewDataResults/save/{window}/{i}/{i}_{window}.csv')
    vars()[i+'_'+window].rename(columns={vars()[i+'_'+window].columns[0]: 'ID'}, inplace=True)
    vars()[i+'_'+window].set_index('ID', inplace=True)

In [ ]:
svd1_10secwindows

# ----------------------------------------------------------------------------------------------------------